In [1]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('AUCs.txt', delimiter='\t')

In [5]:
data.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [12]:
cols =[]

In [13]:
for index, item in enumerate(data.columns):
    if (index == 0):
        cols.append('Datasets')
    else:
        cols.append(item)

In [15]:
data.columns = cols

In [16]:
data.head()

,Datasets,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [17]:
from scipy import stats

In [21]:
from statsmodels.stats.descriptivestats import sign_test

In [20]:
tests = data.columns[1:]

In [30]:
data.iloc[:,2]

0     0.768
1     0.591
2     0.971
3     0.661
4     0.888
5     0.931
6     0.668
7     0.583
8     0.838
9     1.000
10    0.962
11    0.666
12    0.981
13    0.978
Name: C4.5+m, dtype: float64

In [35]:
stats.wilcoxon(data.iloc[:,2], data.iloc[:,3])

c:\program files\python37\lib\site-packages\scipy\stats\morestats.py:2957: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


WilcoxonResult(statistic=17.0, pvalue=0.046332729793395394)

In [49]:
from itertools import combinations

In [72]:
combo = combinations([1,2,3,4], 2)

In [73]:
res = []
for i,j in combo:
    res_tmp = []
    res_tmp.append(data.columns[i])
    res_tmp.append(data.columns[j])
    res_tmp.append(stats.wilcoxon(data.iloc[:,i], data.iloc[:,j])[1])
    res.append(res_tmp)

c:\program files\python37\lib\site-packages\scipy\stats\morestats.py:2957: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [74]:
print(res)

[['C4.5', 'C4.5+m', 0.01075713311978963], ['C4.5', 'C4.5+cf', 0.861262330095348], ['C4.5', 'C4.5+m+cf', 0.015906444101703374], ['C4.5+m', 'C4.5+cf', 0.046332729793395394], ['C4.5+m', 'C4.5+m+cf', 0.3278256758446406], ['C4.5+cf', 'C4.5+m+cf', 0.022909099354356588]]


In [98]:
data_2 = np.array(res)

In [99]:
data_2

array([['C4.5', 'C4.5+m', '0.01075713311978963'],
       ['C4.5', 'C4.5+cf', '0.861262330095348'],
       ['C4.5', 'C4.5+m+cf', '0.015906444101703374'],
       ['C4.5+m', 'C4.5+cf', '0.046332729793395394'],
       ['C4.5+m', 'C4.5+m+cf', '0.3278256758446406'],
       ['C4.5+cf', 'C4.5+m+cf', '0.022909099354356588']], dtype='<U20')

In [100]:
data_p_values = pd.DataFrame(data_2, columns=['First_method', 'Second_method', 'p'])

In [101]:
data_p_values.head()

,First_method,Second_method,p
0,C4.5,C4.5+m,0.01075713311978963
1,C4.5,C4.5+cf,0.861262330095348
2,C4.5,C4.5+m+cf,0.015906444101703374
3,C4.5+m,C4.5+cf,0.046332729793395394
4,C4.5+m,C4.5+m+cf,0.3278256758446406


In [110]:
data_p_values.p = data_p_values.p.astype(float)

In [109]:
for i in data_p_values.p.ravel():
    print(float(i) < 0.05)

True
False
True
True
False
True


In [81]:
from statsmodels.sandbox.stats.multicomp import multipletests

In [86]:
data_p_values.p.values

array(['0.01075713311978963', '0.861262330095348', '0.015906444101703374',
       '0.046332729793395394', '0.3278256758446406',
       '0.022909099354356588'], dtype=object)

In [111]:
reject, p_corrected, _, __ = multipletests(data_p_values.p, alpha=0.05, method='holm')

In [113]:
print(p_corrected)

[0.0645428  0.86126233 0.07953222 0.13899819 0.65565135 0.0916364 ]


In [114]:
reject_2, p_corrected_2, _, __ = multipletests(data_p_values.p, alpha=0.05,
                                              method='fdr_bh')

In [115]:
print(reject_2)

[ True False  True False False  True]
